In [18]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
import tensorflow as tf
from IPython.display import display, Image
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle
from PIL import Image
from six.moves import range


In [19]:
pickle_file= '/input/banglaIsolated_clean.pickle'

with open(pickle_file, 'rb') as f:
    save=pickle.load(f)
    train_dataset=save['train_dataset']
    train_labels=save['train_labels']
    valid_dataset=save['valid_dataset_clean']
    valid_labels=save['valid_labels']
    test_dataset=save['test_dataset_clean']
    test_labels=save['test_labels']
    del save #hint to help gc to free up memory 
    print('training set', train_dataset.shape,train_labels.shape)
    print('validation set', valid_dataset.shape, valid_labels.shape)
    print('test set', test_dataset.shape,test_labels.shape)

training set (139000, 28, 28) (139000,)
validation set (10000, 28, 28) (10000,)
test set (10000, 28, 28) (10000,)


In [20]:
image_size=28
num_labels=84
num_channels=1 

def reformat(dataset, labels):
    dataset=dataset.reshape((-1,image_size,image_size,num_channels)).astype(np.float32)
    # Map 1 to [0.0,1.0,0.0....], 2 to [0.0,0.0,1.0.....]
    labels=(np.arange(num_labels) ==labels[:,None]).astype(np.float32)
    return dataset,labels
train_dataset, train_labels= reformat(train_dataset, train_labels)
valid_dataset, valid_labels=reformat(valid_dataset, valid_labels)
test_dataset, test_labels =reformat(test_dataset, test_labels)
print( 'training set', train_dataset.shape,train_labels.shape)
print('validation set', valid_dataset.shape,valid_labels.shape)
print('test set', test_dataset.shape,test_labels.shape)

training set (139000, 28, 28, 1) (139000, 84)
validation set (10000, 28, 28, 1) (10000, 84)
test set (10000, 28, 28, 1) (10000, 84)


In [21]:
#returns accuracy of model
def accuracy(target,predictions):
    return(100.0*np.sum(np.argmax(target,1) == np.argmax(predictions,1))/target.shape[0])

In [5]:
#need to batch the test data because running low on memory
class test_batchs:
    def __init__(self,data):
        self.data = data
        self.batch_index = 0
    def nextBatch(self,batch_size):
        if (batch_size+self.batch_index) > self.data.shape[0]:
            print ("batch sized is messed up")
        batch = self.data[self.batch_index:(self.batch_index+batch_size),:,:,:]
        self.batch_index= self.batch_index+batch_size
        return batch

#set the test batchsize
test_batch_size = 100

In [22]:
#use os to get our current working directory so we can save variable
file_path = os.getcwd()+'/model.ckpt'

In [23]:
def deeper_inception_conv_net():
    
    batch_size = 50
    patch_size1 = 3
    patch_size2 = 5
    depth = 16
    depth1 = 32
    depth2 = 16
    depth3 = 8
    concat_depth = 72 #concat_depth = depth2+depth3+depth3+depth2
    num_hidden = 1024
    num_hidden2 = 512
    keep_prob = 0.5
    decay_step = 2000
    base = 0.9

    graph = tf.Graph()

    with graph.as_default():

      # Input data.
      tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size, image_size, num_channels))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
      tf_valid_dataset = tf.constant(valid_dataset)
      tf_test_dataset = tf.constant(test_dataset)

      # Variables.
      layer1_weights = tf.Variable(tf.truncated_normal(
          [patch_size1, patch_size1, num_channels, depth], stddev=0.3))
      layer1_biases = tf.Variable(tf.zeros([depth]))
      layer2_weights = tf.Variable(tf.truncated_normal(
          [patch_size2, patch_size2, depth, depth1], stddev=0.05))
      layer2_biases = tf.Variable(tf.constant(0.0, shape=[depth1]))
      layer3_weights = tf.Variable(tf.truncated_normal(
          [((image_size + 3) // 4) * ((image_size + 3) // 4) * concat_depth, num_hidden], stddev=0.05))
      layer3_biases = tf.Variable(tf.constant(0.0, shape=[num_hidden]))
      layer4_weights = tf.Variable(tf.truncated_normal(
          [num_hidden, num_hidden2], stddev=0.01))
      layer4_biases = tf.Variable(tf.constant(0.0, shape=[num_hidden2]))
      layer5_weights = tf.Variable(tf.truncated_normal(
          [num_hidden2, num_labels], stddev=0.01))
      layer5_biases = tf.Variable(tf.constant(0.0, shape=[num_labels]))
        
      inception1x1_weights = tf.Variable(tf.truncated_normal(
          [1, 1, depth1, depth2], stddev=0.25))
      inception1x1_biases = tf.Variable(tf.constant(0.0, shape=[depth2]))
      inception3x3_weights = tf.Variable(tf.truncated_normal(
          [patch_size1, patch_size1, depth2, depth3], stddev=0.05))
      inception3x3_biases = tf.Variable(tf.constant(0.0, shape=[depth3]))
      inception5x5_weights = tf.Variable(tf.truncated_normal(
          [patch_size2, patch_size2, depth2, depth3], stddev=0.08))
      inception5x5_biases = tf.Variable(tf.constant(0.0, shape=[depth3]))
    
      inception1x1_post_mxpool_wts = tf.Variable(tf.truncated_normal(
          [1, 1, depth1, depth2], stddev=0.04))
      post_maxpool_biases = tf.Variable(tf.constant(0.0, shape=[depth2]))
      
        
      ##For inception module2
      inception1x1_weights_mod2 = tf.Variable(tf.truncated_normal(
          [1, 1, 48, 24], stddev=0.25))
      inception1x1_biases_mod2 = tf.Variable(tf.constant(0.0, shape=[24]))
      inception3x3_weights_mod2 = tf.Variable(tf.truncated_normal(
          [patch_size1, patch_size1, 24, 12], stddev=0.05))
      inception3x3_biases_mod2 = tf.Variable(tf.constant(0.0, shape=[12]))
      inception5x5_weights_mod2 = tf.Variable(tf.truncated_normal(
          [patch_size2, patch_size2, 24, 12], stddev=0.08))
      inception5x5_biases_mod2 = tf.Variable(tf.constant(0.0, shape=[12]))
    
      inception1x1_post_mxpool_wts_mod2 = tf.Variable(tf.truncated_normal(
          [1, 1, 48, 24], stddev=0.04))
      post_maxpool_biases_mod2 = tf.Variable(tf.constant(0.0, shape=[24]))
      #End of inception module2
      
      global_step = tf.Variable(0, trainable = False)  # count the number of steps taken.
      learning_rate = tf.train.exponential_decay(0.005, global_step, decay_step, base)

        
      # Model.
      def model(data, useDropout):
        print(data)
        conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        print(conv)
        max_pooled = tf.nn.max_pool(conv, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        print(max_pooled)
        hidden = tf.nn.relu(max_pooled + layer1_biases)
        print(hidden)
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
        print(conv)
        max_pooled = tf.nn.max_pool(conv, [1, 2, 2, 1], [1, 1, 1, 1], padding='SAME')
        print(max_pooled)
        hidden = tf.nn.relu(max_pooled + layer2_biases)
        print(hidden)
        print("b4 inception")
        print(inception1x1_weights)
        print(inception3x3_weights)
        print(inception5x5_weights)
        inception1x1_conv = tf.nn.conv2d(hidden, inception1x1_weights, [1, 1, 1, 1], padding='SAME')
        inception1x1_relu = tf.nn.relu(inception1x1_conv + inception1x1_biases)
        
        inception3x3_conv = tf.nn.conv2d(inception1x1_relu, inception3x3_weights, [1, 1, 1, 1], padding='SAME')
        inception3x3_relu = tf.nn.relu(inception3x3_conv + inception3x3_biases)
        
        inception5x5_conv = tf.nn.conv2d(inception1x1_relu, inception5x5_weights, [1, 1, 1, 1], padding='SAME')
        inception5x5_relu = tf.nn.relu(inception5x5_conv + inception5x5_biases)
        
        inception3x3_maxpool = tf.nn.max_pool(hidden, [1, 3, 3, 1], [1, 1, 1, 1], padding='SAME')
        inception1x1_post_maxpool = tf.nn.conv2d(inception3x3_maxpool, inception1x1_post_mxpool_wts, [1, 1, 1, 1], padding='SAME')
        inception1x1_post_maxpool = tf.nn.relu(inception1x1_post_maxpool + post_maxpool_biases)
        print(inception1x1_relu, inception3x3_relu, inception5x5_relu, inception1x1_post_maxpool)
        concat_filter = tf.concat([inception1x1_relu, inception3x3_relu, inception5x5_relu, inception1x1_post_maxpool],3)
        print("after inception")
        print(concat_filter)
        
        print("b4 inception 2")
        
        inception1x1_conv = tf.nn.conv2d(concat_filter, inception1x1_weights_mod2, [1, 1, 1, 1], padding='SAME')
        inception1x1_relu = tf.nn.relu(inception1x1_conv + inception1x1_biases_mod2)
        
        inception3x3_conv = tf.nn.conv2d(inception1x1_relu, inception3x3_weights_mod2, [1, 1, 1, 1], padding='SAME')
        inception3x3_relu = tf.nn.relu(inception3x3_conv + inception3x3_biases_mod2)
        
        inception5x5_conv = tf.nn.conv2d(inception1x1_relu, inception5x5_weights_mod2, [1, 1, 1, 1], padding='SAME')
        inception5x5_relu = tf.nn.relu(inception5x5_conv + inception5x5_biases_mod2)
        
        inception3x3_maxpool = tf.nn.max_pool(concat_filter, [1, 3, 3, 1], [1, 1, 1, 1], padding='SAME')
        inception1x1_post_maxpool = tf.nn.conv2d(inception3x3_maxpool, inception1x1_post_mxpool_wts_mod2, [1, 1, 1, 1], padding='SAME')
        inception1x1_post_maxpool = tf.nn.relu(inception1x1_post_maxpool + post_maxpool_biases_mod2)
        print(inception1x1_relu, inception3x3_relu, inception5x5_relu, inception1x1_post_maxpool)
        concat_filter = tf.concat([inception1x1_relu, inception3x3_relu, inception5x5_relu, inception1x1_post_maxpool],3)
        print("after inception 2")
        print(concat_filter)
        
        concat_maxpooled = tf.nn.max_pool(concat_filter, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        print(concat_maxpooled)
        shape = concat_maxpooled.get_shape().as_list()
        
        reshape = tf.reshape(concat_maxpooled, [shape[0], shape[1] * shape[2] * shape[3]])
        print(reshape)
        if useDropout == 1:
            dropout_layer2 = tf.nn.dropout(tf.nn.relu(reshape), keep_prob)
        else:
            dropout_layer2 = tf.nn.relu(reshape)
        print(dropout_layer2)
        print("1st fc")
        hidden = tf.nn.relu(tf.matmul(dropout_layer2, layer3_weights) + layer3_biases)
        print(hidden)
        print("2nd fc")
        hidden = tf.nn.relu(tf.matmul(hidden, layer4_weights) + layer4_biases)
        print(hidden)
        print("after fully Connected")
        return tf.matmul(hidden, layer5_weights) + layer5_biases

      # Training computation.
      logits = model(tf_train_dataset, 1)
      loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))

      # Optimizer.
      optimizer = tf.train.AdamOptimizer(0.001).minimize(loss, global_step=global_step)

      # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(model(tf_train_dataset, 0))
      valid_prediction = tf.nn.softmax(model(tf_valid_dataset, 0))
      test_prediction = tf.nn.softmax(model(tf_test_dataset, 0))
      saver = tf.train.Saver()
        
        
    num_steps = 30001

    with tf.Session(graph=graph) as session:
      tf.initialize_all_variables().run()
      print('Initialized')
      for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
          print('Minibatch loss at step %d: %f' % (step, l))
          print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
          #print(tf.Print(layer1_weights, [layer1_weights]).eval())
          print('Validation accuracy: %.1f%%' % accuracy(
            valid_prediction.eval(), valid_labels))
      print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels)) 
    
deeper_inception_conv_net()


Tensor("Placeholder:0", shape=(50, 28, 28, 1), dtype=float32)
Tensor("Conv2D:0", shape=(50, 28, 28, 16), dtype=float32)
Tensor("MaxPool:0", shape=(50, 14, 14, 16), dtype=float32)
Tensor("Relu:0", shape=(50, 14, 14, 16), dtype=float32)
Tensor("Conv2D_1:0", shape=(50, 14, 14, 32), dtype=float32)
Tensor("MaxPool_1:0", shape=(50, 14, 14, 32), dtype=float32)
Tensor("Relu_1:0", shape=(50, 14, 14, 32), dtype=float32)
b4 inception
Tensor("Variable_10/read:0", shape=(1, 1, 32, 16), dtype=float32)
Tensor("Variable_12/read:0", shape=(3, 3, 16, 8), dtype=float32)
Tensor("Variable_14/read:0", shape=(5, 5, 16, 8), dtype=float32)
Tensor("Relu_2:0", shape=(50, 14, 14, 16), dtype=float32) Tensor("Relu_3:0", shape=(50, 14, 14, 8), dtype=float32) Tensor("Relu_4:0", shape=(50, 14, 14, 8), dtype=float32) Tensor("Relu_5:0", shape=(50, 14, 14, 16), dtype=float32)
after inception
Tensor("concat:0", shape=(50, 14, 14, 48), dtype=float32)
b4 inception 2
Tensor("Relu_6:0", shape=(50, 14, 14, 24), dtype=float32) 

ResourceExhaustedError: OOM when allocating tensor with shape[10000,28,28,16]
	 [[Node: Conv2D_20 = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/gpu:0"](Const, Variable/read)]]

Caused by op u'Conv2D_20', defined at:
  File "/usr/local/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/local/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python2.7/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    sup

Model initialized.


ResourceExhaustedError: OOM when allocating tensor with shape[10000,28,28,128]
	 [[Node: concat_4 = ConcatV2[N=4, T=DT_FLOAT, Tidx=DT_INT32, _device="/job:localhost/replica:0/task:0/gpu:0"](add_14, add_17, add_18, add_19, concat_4/axis)]]
	 [[Node: Softmax/_19 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_47_Softmax", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op u'concat_4', defined at:
  File "/usr/local/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/local/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in 